In [1]:
import requests
import time
from geopy.distance import geodesic
import datetime
import geocoder
import csv

stationDict = {}
station2016Dict={}
stationJson = requests.get('https://gbfs.citibikenyc.com/gbfs/es/station_information.json').json()


In [2]:
#put bike stations and location into local dict and csv file
def createFullStationCSV():
    with open('bikeShareData.csv', 'w', newline='') as f:
        fieldnames = ['id', 'name', 'lat', 'lon']
        thewriter = csv.DictWriter(f, fieldnames = fieldnames)
        thewriter.writeheader()

        for each in stationJson['data']['stations']:
            #print(each['name'])
            id = each['station_id']
            thewriter.writerow({'id' : id, 'name' : each['name'], 'lat' : each['lat'], 'lon' :  each['lon']})
            stationDict[id] = [each['name'], each['lat'], each['lon']]

In [7]:
def create2016StationCSV(fileName):
    count = 1
    with open(fileName) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        next(readCSV)
        for row in readCSV:
            #print(row[0], row[5], row[6])
            #print(valueS)
            lat = row[5]
            lon =  row[6]
            station2016Dict[row[3]] = [row[3],row[5],row[6],0]
            
        print("Total stations read: ",len(station2016Dict))
            
    interval = len(station2016Dict)/20
    baseInterval = interval
    progress = 5
    with open('2016Stations.csv', 'w', newline='') as f:
        fieldnames = ['id', 'lat', 'lon', 'zip']
        print("geocoding...")
        thewriter = csv.writer(f)
        thewriter.writerow(fieldnames)
        print("progress: ", end="")
        for key, value in station2016Dict.items():
            g = geocoder.arcgis([value[1], value[2]], method='reverse')
            value[3] = g.postal
            if count > interval:
                print(progress+"%", end=" ")
                progress +=5
                interval += baseInterval
            thewriter.writerow(value)
            time.sleep(.01)
            count+=1
        print(progress+"%", end=" ")

In [8]:
create2016StationCSV("201612-citibike-tripdata.csv")

Total stations read:  609
geocoding...
progress: 

TypeError: unsupported operand type(s) for +: 'int' and 'str'

look for datasets here https://s3.amazonaws.com/tripdata/index.html